In [1]:
pserver.stop()

NameError: name 'pserver' is not defined

In [1]:
from parameter_server import *
pserver = ParameterServer()

Parameter server started on ('127.0.0.1', 9011)
Serving on ('127.0.0.1', 14001)
Sending registering as listener request
Register listener 127.0.0.1:9010


# Usage of DiskBufferWriter

In [2]:
from disk_buffer_writer import *

In [3]:
background = DiskBufferWriter('background','background.json', 'bin/')
background.expose_to_network(pserver)

In [23]:
import random, math

background.data = [random.random() for i in range(3)]

# Cache to compensate for possible time writing overlaps

In [24]:
dataWriter = DiskBufferWriter('sine_data','output.json', 'bin/')
dataWriter.expose_to_network(pserver)
dataWriter.allow_cache(10)

In [26]:
import random, math

numpoints = random.randint(4,20)

rnums = [random.random() for i in range(15)]
rnums.sort()
dataWriter.data = {'random' : [-1.0 + (n* 2.0) for n in rnums],
                   'sines' : [math.sin(r* 2 * math.pi) for r in rnums]
                  }

In [27]:
dataWriter.data

{'random': [-0.9412092207560403,
  -0.8504762532285022,
  -0.7574847573971542,
  -0.49576817838920983,
  -0.30495895277044305,
  -0.29423214711140977,
  -0.2479154443904561,
  -0.0923277646414109,
  -0.02546937391850146,
  0.7986102958187049,
  0.8036616991207439,
  0.8042775383928251,
  0.8392086013678759,
  0.8451839791684843,
  0.9769530046355408],
 'sines': [0.18364838101597708,
  0.4526568737883265,
  0.6902858883410149,
  0.9999116273135992,
  0.8180755615824071,
  0.7982339508727302,
  0.7024609331838534,
  0.28600610394599907,
  0.07992904590577181,
  -0.5913117139216223,
  -0.5784399853192953,
  -0.5768607075090014,
  -0.48393090223852614,
  -0.46741893054929523,
  -0.07234102613838435]}

In [28]:
import time
for i in range(20):
    numpoints = random.randint(4,20)

    rnums = [random.random() for i in range(15)]
    rnums.sort()
    dataWriter.data = {'random' : [-1.0 + (n* 2.0) for n in rnums],
                       'sines' : [math.sin(r* 2 * math.pi) for r in rnums]
                      }
    time.sleep(0.1)

Now let's send changes really fast to test disk caching. (It may fail on your system if your drive access is not fast enough, causing the app to crash)

In [29]:
import time
dataWriter.allow_cache(100)

for i in range(200):
    numpoints = random.randint(4,20)

    rnums = [random.random() for i in range(15)]
    rnums.sort()
    dataWriter.data = {'random' : [-1.0 + (n* 2.0) for n in rnums],
                       'sines' : [math.sin(r* 2 * math.pi) for r in rnums]
                      }
    time.sleep(0.001)

This might leave a large cache, so cleanup:

In [30]:
dataWriter.cleanup()

If you need guaranteed safety, use file locking:

In [31]:

# No cache but file locking
dataWriter.allow_cache(-1)
dataWriter.use_file_lock()

import time
for i in range(10000):
    numpoints = random.randint(4,20)

    rnums = [random.random() for i in range(15)]
    rnums.sort()
    dataWriter.data = {'random' : [-1.0 + (n* 2.0) for n in rnums],
                       'sines' : [math.sin(r* 2 * math.pi) for r in rnums]
                      }
    time.sleep(0.00001) # Very fast updates

If this is still not enough, you can have the data writer wait for read acknowledge from app.

Currently not implemented.

In [ ]:
pserver.stop()